# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [23]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '/Users/justin/Desktop/100daysML/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_num = df_train.shape[0]  ## 891
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()
train_num

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



891

# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [37]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
LEncoder = LabelEncoder()
df_temp = pd.DataFrame()
for c in df.columns:
    
    df_temp[c] = LEncoder.fit_transform(df[c])
train_X = df_temp[:train_num]

lg_estimator = LogisticRegression()
# lg_estimator.fit(train_X, train_Y)
cv_score = cross_val_score(lg_estimator,train_X,train_Y,cv=5).mean()

print(f' Using {train_num} row of data in our training set, we used 5 fold\
cross validation in logistic regression, getting {cv_score} accuracy in our model')


 Using 891 row of data in our training set, we used 5 foldcross validation in logistic regression, getting 0.780004837244799 accuracy in our model


/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to '

In [61]:
# Mean encoding + Logistic regression
# use mean encoding in every discrete variable for different classes
data_mean = pd.concat([df[:train_num],train_Y],axis = 1)
data_mean = data_mean.drop(['Name'],axis =1)
# grouping by names doesn't have much meaning, so I'll drop Names for the regression
# data.groupby("Name")['Survived'].mean()

data_mean


,Sex,Ticket,Cabin,Embarked,Survived
0,male,A/5 21171,None,S,0
1,female,PC 17599,C85,C,1
2,female,STON/O2. 3101282,None,S,1
3,female,113803,C123,S,1
4,male,373450,None,S,0
5,male,330877,None,Q,0
6,male,17463,E46,S,0
7,male,349909,None,S,0
8,female,347742,None,S,1
9,female,237736,None,C,1


In [84]:
## curious about chi-square test results

from scipy.stats import chi2_contingency

for c in data_mean.columns:
    if c != "Survived":
        two_by_two_table = pd.crosstab(data_mean[c],data_mean['Survived'])
        chi_square_results = chi2_contingency(two_by_two_table)
        print(f'The chi square value of {c} and Survived is {chi_square_results[0]}, leading\
        to a p-value of {chi_square_results[1]}, therefore we can conclude that the\
        the hypothesis that these two variables have a relationship is {chi_square_results[1]<0.05}\
        under 0.05 confidence. \n')
# all four variables have a low p-value when conducting chi-square tests, thus we can
# conclude that they all are significant



The chi square value of Sex and Survived is 260.71702016732104, leading        to a p-value of 1.1973570627755645e-58, therefore we can conclude that the        the hypothesis that these two variables have a relationship is True        under 0.05 confidence. 

The chi square value of Ticket and Survived is 766.5697029458893, leading        to a p-value of 0.01152729601163775, therefore we can conclude that the        the hypothesis that these two variables have a relationship is True        under 0.05 confidence. 

The chi square value of Cabin and Survived is 240.99672677470033, leading        to a p-value of 1.617698166309318e-06, therefore we can conclude that the        the hypothesis that these two variables have a relationship is True        under 0.05 confidence. 

The chi square value of Embarked and Survived is 29.670614191394378, leading        to a p-value of 1.618718800945046e-06, therefore we can conclude that the        the hypothesis that these two variables have a relat

In [86]:
for col in df.columns:
    if col != "Name":
        mean_col = data_mean.groupby([col])['Survived'].mean().reset_index()
        mean_col.columns = [col,str(col)+"_mean"]
        data_mean = pd.merge(data_mean,mean_col,on=col, how="left")
        data_mean = data_mean.drop([col],axis =1)

data_mean = data_mean.drop(['Survived'], axis =1)


In [87]:
lg_estimator = LogisticRegression()
cv_score = cross_val_score(lg_estimator,data_mean,train_Y,cv=5).mean()

print(f' Using mean encoding in our training set, we used 5 fold\
cross validation in logistic regression, getting {cv_score} accuracy in our model')



 Using mean encoding in our training set, we used 5 foldcross validation in logistic regression, getting 0.9730773636448428 accuracy in our model


/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to '

In [ ]:
## using mean encoding we got 97% accuracy